In [2]:
import pandas as pd

In [13]:
db = 'TableroInformacion2.xlsx'
fecha_proceso = "2023-06-15"

In [104]:
df_Bloomberg = pd.read_excel(db ,sheet_name='Bloomberg' , dtype = {'Fecha': str})
df_PIP = pd.read_excel(db ,sheet_name='PIP', dtype = {'Fecha': str})

df_SBS_PEN = pd.read_excel(db, sheet_name='SBS PEN', dtype = {'Fecha': str})
df_SBS_USD = pd.read_excel(db, sheet_name='SBS USD', dtype = {'Fecha': str})

df_Caracteristicas = pd.read_excel(db ,sheet_name='Caracteristicas')

df_flujos = pd.read_excel(db ,sheet_name='Flujo')


df_Bloomberg = df_Bloomberg.rename(columns=lambda x: x.strip())
df_PIP = df_PIP.rename(columns=lambda x: x.strip())
df_SBS_PEN = df_SBS_PEN.rename(columns=lambda x: x.strip())
df_SBS_USD = df_SBS_USD.rename(columns=lambda x: x.strip())
df_Caracteristicas = df_Caracteristicas.rename(columns=lambda x: x.strip())
df_flujos = df_flujos.rename(columns=lambda x: x.strip())




In [95]:
def calculo_duracion(fila):
    try:
        duracion = ((fila['Amortización']*fila['Plazo']/365)/((1+fila['Tasa Bono'])**(fila['Plazo']/365)))/(fila['Amortización']/(1+fila['Tasa Bono'])**(fila['Plazo']/365))
    except:
        duracion = 0

    return duracion

def calculo_TIR(fila):

    tir = ((fila['Amortización'])/((1+fila['Tasa Bono'])**(fila['Plazo']/365)))

    return tir

In [67]:
df_flujos.columns

Index(['ISIN', 'Numero Cupon', 'Fecha Vto', 'Fecha Pago', 'Fecha Fijacion',
       'Tasa Bono', 'Amortización'],
      dtype='object')

In [96]:

def get_duracion_and_TIR(fecha_proceso,df_flujos,isin):

    fecha_proceso_dt = pd.to_datetime(fecha_proceso)

    df_flujos_bono = df_flujos[(df_flujos.ISIN==isin) & (df_flujos["Fecha Vto"]>=fecha_proceso_dt)].copy()

    df_flujos_bono.sort_values(by="Fecha Vto",ascending=True,inplace=True)

    df_flujos_bono["Plazo"] =  (df_flujos_bono["Fecha Vto"] - fecha_proceso_dt).dt.days

    df_flujos_bono['Duracion'] = df_flujos_bono.apply(calculo_duracion, axis=1)
    df_flujos_bono['TIR'] = df_flujos_bono.apply(calculo_TIR, axis=1)

    df_flujos_bono_agg = df_flujos_bono.groupby(["ISIN"],as_index=False).agg({'Duracion': "sum", 'TIR': "sum"})
    df_flujos_bono_agg["TIR"] = df_flujos_bono_agg["TIR"]/100

    Duracion = df_flujos_bono_agg.loc[0, 'Duracion']  # Valor en la primera fila, columna1
    TIR = df_flujos_bono_agg.loc[0, 'TIR'] 

    return Duracion , TIR





In [105]:
df_Bloomberg['Fecha'] = pd.to_datetime(df_Bloomberg['Fecha'], format='%Y-%m-%d')

df_PIP["Fecha"] = fecha_proceso
df_PIP['Fecha'] = pd.to_datetime(df_PIP['Fecha'], format='%Y-%m-%d')

df_SBS_PEN['Fecha'] = pd.to_datetime(df_SBS_PEN['Fecha'], format='%Y-%m-%d')
df_SBS_USD['Fecha'] = pd.to_datetime(df_SBS_USD['Fecha'], format='%Y-%m-%d')


In [106]:
#igualando a la fecha de proceso y quitando lo de bloomberg que pertebece a la renta variable
df_Bloomberg = df_Bloomberg[(df_Bloomberg.Fuente!="BLG") & (df_Bloomberg.Fecha==fecha_proceso) ].copy()

df_Bloomberg["Valor"] = df_Bloomberg["Valor"]/100


df_PIP["Fuente"] = "PIP"

df_SBS_PEN["Fuente"] = "SBS"
df_SBS_PEN["P. Limpio (%)"] = df_SBS_PEN["P. Limpio (%)"]/100


df_SBS_USD["Fuente"] = "SBS"
df_SBS_USD["P. Limpio (%)"] = df_SBS_USD["P. Limpio (%)"]/100



In [107]:
ipb_cls =  ['Fecha', 'ISIN', 'Precio','Fuente']

df_input_precio_bono = pd.DataFrame(columns = ipb_cls)


df_temp=pd.DataFrame(df_Bloomberg[["Fecha","Instrumento","Valor","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono = df_input_precio_bono.append(df_temp)

df_temp=pd.DataFrame(df_PIP[["Fecha","ISIN","Precio Limpio %","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono = df_input_precio_bono.append(df_temp)

df_temp=pd.DataFrame(df_SBS_PEN[["Fecha","ISIN","P. Limpio (%)","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono = df_input_precio_bono.append(df_temp)

df_temp=pd.DataFrame(df_SBS_USD[["Fecha","ISIN","P. Limpio (%)","Fuente"]].values, columns=ipb_cls)
df_input_precio_bono = df_input_precio_bono.append(df_temp)

df_input_precio_bono.head()


,Fecha,ISIN,Precio,Fuente
0,2023-06-15,US912810EQ77,1.000664,TRAC
1,2023-06-15,US912810EQ77,1.000664,BVAL
2,2023-06-15,US912810EQ77,0.998906,BGN
3,2023-06-15,US912810FJ26,1.119219,BGN
4,2023-06-15,US912810FJ26,1.119297,BVAL


In [124]:
def set_metodo_nuevo(fila):

    if fila['Pais Emisor']=="PE" :

        # Aplicar la lógica del switch case

        if pd.isna(fila['PIP'])==False:
            return 'PIP'
        elif pd.isna(fila['SBS'])==False:
            return 'SBS'
        else:
            return 'TIR'
        
    elif fila['Pais Emisor']!="PE":

        if pd.isna(fila['BVAL'])==False:
            return 'BVAL'
        else:
            return 'TIR'

In [125]:

def get_TIR_anterior(fecha_proceso,df_flujos,isin):

    fecha_proceso_dt = pd.to_datetime(fecha_proceso)

    df_flujos_bono = df_flujos[(df_flujos.ISIN==isin) & (df_flujos["Fecha Vto"]<fecha_proceso_dt)].copy()

    df_flujos_bono.sort_values(by="Fecha Vto",ascending=True,inplace=True)

    df_flujos_bono["Plazo"] =  (df_flujos_bono["Fecha Vto"] - fecha_proceso_dt).dt.days

   
    df_flujos_bono['TIR'] = df_flujos_bono.apply(calculo_TIR, axis=1)

    df_flujos_bono_agg = df_flujos_bono.groupby(["ISIN"],as_index=False).agg({'TIR': "sum"})
    df_flujos_bono_agg["TIR"] = df_flujos_bono_agg["TIR"]/100


    TIR = df_flujos_bono_agg.loc[0, 'TIR'] 

    return TIR

In [126]:
cls_carac = ["ISIN","Pais Emisor","Tipo Bono"]
df_caract = df_Caracteristicas[cls_carac].copy()
print(df_caract.shape)
for fuente in ["PIP","BVAL","SBS","TIR"]:
    df_precio = df_input_precio_bono[(df_input_precio_bono.Fecha==fecha_proceso) & (df_input_precio_bono.Fuente==fuente)][["ISIN","Precio"]].copy()
    df_precio.rename(columns={"Precio": fuente},  inplace=True )
    df_caract = pd.merge(df_caract,df_precio,left_on="ISIN",right_on="ISIN",how="left")

    if fuente == "TIR":
        df_caract['TIR'] = None
        for index, row in df_caract.iterrows():
            # Obtén el valor actual de 'columna_actualizar'
            isin = row['ISIN']
            try:
                tir = get_TIR_anterior(fecha_proceso,df_flujos,isin)
                # Actualiza el valor en la columna correspondiente     
                df_caract.at[index, 'TIR'] = tir
            except:
                continue

df_caract['MetodoValNuevo'] = df_caract.apply(set_metodo_nuevo, axis=1)

df_caract.head(10)

(31, 3)


,ISIN,Pais Emisor,Tipo Bono,PIP,BVAL,SBS,TIR,MetodoValNuevo
0,PEP12100D243,PE,Bono Corporativo,NaN,NaN,1.01315,None,SBS
1,PEP58501M073,PE,Bono Soberano,0.991568,NaN,0.96688,None,PIP
2,PEP72760M010,PE,Bono Corporativo,0.91496,NaN,NaN,None,PIP
3,PEP73125M049,US,Bono Corporativo,0.987375,NaN,NaN,None,TIR
4,PEP73125M056,US,Bono Corporativo,0.964907,NaN,NaN,None,TIR
5,PEP73125M064,PE,Bono Soberano,0.901541,NaN,NaN,None,PIP
6,US912810EQ77,US,Bono Tesoro,NaN,1.000664,NaN,0.0,BVAL
7,US912810FJ26,US,Bono Tesoro,NaN,1.119297,NaN,None,BVAL
8,US912810FT08,US,Bono Tesoro,0.901541,1.085625,NaN,0.0,BVAL
9,US912810PU60,US,Bono Tesoro,NaN,1.13875,NaN,0.0,BVAL


In [91]:
df_Caracteristicas["Duracion"]=None
df_Caracteristicas["TIR"]=None


for index, row in df_Caracteristicas.iterrows():
    try:
        # Obtén el valor actual de 'columna_actualizar'
        isin = row['ISIN']
        dur, tir = get_duracion_and_TIR(fecha_proceso,df_flujos,isin)
        # Actualiza el valor en la columna correspondiente
        df_Caracteristicas.at[index, 'Duracion'] = dur
        df_Caracteristicas.at[index, 'TIR'] = tir
    except:
        continue

df_Caracteristicas

,ISIN,Pais Emisor,Tipo Bono,PIP,BVAL,SBS,MetodoValNuevo,Duracion,TIR
0,PEP12100D243,PE,Bono Corporativo,NaN,NaN,1.01315,PIP,None,None
1,PEP58501M073,PE,Bono Soberano,0.991568,NaN,0.96688,PIP,None,None
2,PEP72760M010,PE,Bono Corporativo,0.91496,NaN,NaN,PIP,None,None
3,PEP73125M049,US,Bono Corporativo,0.987375,NaN,NaN,BVAL,None,None
4,PEP73125M056,US,Bono Corporativo,0.964907,NaN,NaN,BVAL,None,None
5,PEP73125M064,PE,Bono Soberano,0.901541,NaN,NaN,PIP,None,None
6,US912810EQ77,US,Bono Tesoro,NaN,1.000664,NaN,BVAL,0.169863,0.989755
7,US912810FJ26,US,Bono Tesoro,NaN,1.119297,NaN,BVAL,None,None
8,US912810FT08,US,Bono Tesoro,0.901541,1.085625,NaN,BVAL,12.679452,0.57229
9,US912810PU60,US,Bono Tesoro,NaN,1.13875,NaN,BVAL,13.926027,0.506894


In [41]:
df_Caracteristicas[df_Caracteristicas["Pais Emisor"]=="PE"]["Tipo Bono"].value_counts()

Certificado Deposito    2
Bono Soberano           2
Bono Corporativo        2
Name: Tipo Bono, dtype: int64

In [ ]:
ipb_cls =  ['ISIN', 'PrecioPIP', 'PrecioSBS', 'PrecioBVAL','TasaANT','MetodoValNuevo']
df_temporal_vector = pd.DataFrame(columns = ipb_cls)